In [1]:
import os

from os.path import join as ospj
from os.path import expanduser
from munch import Munch as mch
import numpy as np

from ds import prepare_coco_dataloaders, prepare_flickr_dataloaders, prepare_cub_dataloaders, prepare_flo_dataloaders

from utils import *
from networks import *
from train_probVLM import *

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'ProbVLM'

In [ ]:
dataset = 'coco' # coco or flickr
data_dir = ospj('../../dataset', dataset) # e.g. ospj(expanduser('~'), 'Documents', 'jm', 'data', dataset)
dataloader_config = mch({
    'batch_size': 32,
    'random_erasing_prob': 0.,
    'traindata_shuffle': True
})
loaders = load_data_loader(dataset, data_dir, dataloader_config, num_workers=12)
coco_train_loader, coco_valid_loader, coco_test_loader = loaders['train'], loaders['val'], loaders['test']

creating index...
index created!
Loading COCO Caption: n_images 113287 n_captions 566435...
loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
Loading COCO Caption: n_images 1000 n_captions 5000...
loading annotations into memory...
Done (t=0.41s)
creating index...
index created!
Loading COCO Caption: n_images 5000 n_captions 25000...


In [ ]:
# clip_net = load_model('cuda')
CLIP_Net = load_model(device='cuda', model_path=None)
ProbVLM_Net = BayesCap_for_CLIP(
    inp_dim=512,
    out_dim=512,
    hid_dim=256,
    num_layers=3,
    p_drop=0.05,
)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 4 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [ ]:
train_ProbVLM(
    CLIP_Net,
    ProbVLM_Net,
    coco_train_loader,
    coco_valid_loader,
    Cri = TempCombLoss(),
    device='cuda',
    dtype=torch.cuda.FloatTensor,
    init_lr=8e-5,
    num_epochs=500,
    eval_every=5,
    ckpt_path='../../models/',
    T1=1e0,
    T2=1e-4
)

  0%|          | 0/17702 [00:00<?, ?batch/s]

Epoch 0:  11%|█▏        | 2001/17702 [12:54<1:41:19,  2.58batch/s, loss=2.45]


Avg. loss: 0.3949558953756182


Validating ...: 100%|██████████| 157/157 [01:04<00:00,  2.43batch/s]


Avg. MSE: 1.699514389038086 | Avg. MAE: 1.0173567533493042
current score: 1.0173567533493042 | Last best score: 100000000.0


...

Avg. loss: 0.1703449706615883


Epoch 4:  11%|█▏        | 2001/17702 [12:01<1:34:22,  2.77batch/s, loss=1.34]


Avg. loss: 0.1505195981175896


Epoch 5:   1%|          | 154/17702 [00:58<1:51:29,  2.62batch/s, loss=1.3] 


KeyboardInterrupt: 